In [3]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip insatll request

ERROR: unknown command "insatll" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [255]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [256]:
# parse the data and make the dataframe
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, "lxml")
table = soup.find("table", class_="wikitable")
link = table.find_all("tr")
link[1].td.text

postal = [] #list of postal code
borough = [] # list of borough
neighbor = [] # list of neighborhood

for i in range(1,181):
    postal.append(link[i].find_all("td")[0].text.replace("\n",""))
    
for i in range(1,181):
    borough.append(link[i].find_all("td")[1].text.replace("\n",""))

for i in range(1,181):
    neighbor.append(link[i].find_all("td")[2].text.replace("\n",""))



In [265]:
# Make dataframe
df = pd.DataFrame({
    "Postal code": postal,
    "Borough": borough,
    "Neighborhood": neighbor
                  })
# Create a list of not assigned index column
row_not = []
for x, y in enumerate(df["Borough"]):
    if y == "Not assigned":
        row_not.append(x)

# Drop not assign row
df.drop(row_not, inplace=True)
df.reset_index(inplace=True)
#Change "/" to ","
df["Neighborhood"] = df["Neighborhood"].str.replace("/", ",")
df = df.drop(["index"], axis=1)
df.shape

(103, 3)

In [266]:
df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [331]:
df_1 = pd.read_csv("Geospatial_Coordinates.csv")

row_ind = []
for x in df["Postal code"]:
    for i, y in enumerate(df_1["Postal Code"]):
        if x==y:
            row_ind.append(i)

df_2 = df_1.loc[row_ind, ["Latitude", "Longitude"]]
df_2 = df_2.reset_index().drop(["index"], axis=1)
df_2 = pd.concat([df, df_2], axis=1)

In [333]:
toronto_list = ["Central Toronto", "East Toronto", "Downtown Toronto", "West Toronto"]
central = df_2[df["Borough"]==toronto_list[0]]
east = df_2[df_2["Borough"]==toronto_list[1]]
downtown = df_2[df_2["Borough"]==toronto_list[2]]
west = df_2[df_2["Borough"]==toronto_list[3]]
df_toronto = pd.concat([central, east, downtown, west], axis=0)
df_toronto.reset_index(inplace=True)
df_toronto = df_toronto.drop(["index", "Postal code"], axis=1)
df_toronto

,Borough,Neighborhood,Latitude,Longitude
0,Central Toronto,Lawrence Park,43.728020,-79.388790
1,Central Toronto,Roselawn,43.711695,-79.416936
2,Central Toronto,Davisville North,43.712751,-79.390197
3,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,Central Toronto,North Toronto West,43.715383,-79.405678
5,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678
6,Central Toronto,Davisville,43.704324,-79.388790
7,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160
8,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686412,-79.400049
9,East Toronto,The Beaches,43.676357,-79.293031


In [343]:

# create map of Manhattan using latitude and longitude values
longitude = -79.347015
latitude = 43.651070
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [387]:
import json
from pandas.io.json import json_normalize 
CLIENT_ID = '0E2W0K50NHZMGMNW3LP2NDJNZWBAMDHGGPTZA40IP0KGWAAJ' 
CLIENT_SECRET = 'NJSPIXE2GDYUT4YPSMXDF1HTWPR54YYYGG4U5HSTIDVN025D'
VERSION = '20180605'

neighborhood_latitude = df_toronto.loc[0, "Latitude"]
neighborhood_longitude = df_toronto.loc[0, "Longitude"]

url = "https://api.foursquare.com/v2/venues/explore?client_id={} &client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    neighborhood_latitude,
    neighborhood_longitude,
    VERSION,
    500,
    100
)

result = requests.get(url).json()
venues = result["response"]["groups"][0]["items"]# get the venues
nearby_venues = json_normalize(venues) #turn the venues data to dataframe
use_venue = nearby_venues[["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]]
for x in range(0,3):
    use_venue.loc[x,"venue.categories"] = use_venue.loc[x,"venue.categories"][0]["name"]
use_venue

/Users/buinhatquang/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [389]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)




toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )




Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex , North Midtown , Yorkville
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Business reply mail Processing CentrE
Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Stn A PO Boxes
St. James Town , Cabbagetown
First Canadian Place , Underground city
Church and Wellesley
Dufferin , Dovercourt Village
Li

In [469]:
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix="", prefix_sep="")
toronto_onehot["Neighborhood"] = df_toronto["Neighborhood"]
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Brockton , Parkdale Village , Exhibition Place",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Business reply mail Processing CentrE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"CN Tower , King and Spadina , Railway Lands , ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central Bay Street,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Christie,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Church and Wellesley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Commerce Court , Victoria Hotel",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [464]:
top_venues = 10
indicators = ["st", "nd", "rd"]
columns = ["Neighborhood"]

for ind in range(top_venues):
    try:
        columns.append("{}{} Most famous venue".format(ind+1, indicator[ind]))
    except:
        columns.append("{}th Most famous venue".format(ind+1))
        
neighborhood_sort = pd.DataFrame(columns=columns)
neighborhood_sort["Neighborhood"] = toronto_grouped["Neighborhood"]

for ind in range(len(toronto_grouped)):
    neighborhood_sort.iloc[ind, 1:] = toronto_grouped.iloc[ind, 1:].sort_values(ascending=False).iloc[0:10].index
neighborhood_sort

,Neighborhood,1th Most famous venue,2th Most famous venue,3th Most famous venue,4th Most famous venue,5th Most famous venue,6th Most famous venue,7th Most famous venue,8th Most famous venue,9th Most famous venue,10th Most famous venue
0,Berczy Park,Chinese Restaurant,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Clothing Store,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,Business reply mail Processing CentrE,Park,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Sporting Goods Shop,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,Central Bay Street,Salon / Barbershop,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,Christie,Diner,Yoga Studio,Farmers Market,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
6,Church and Wellesley,Seafood Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
7,"Commerce Court , Victoria Hotel",Coffee Shop,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
8,Davisville,Breakfast Spot,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,Davisville North,Bus Line,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [517]:
from sklearn.cluster import KMeans
neighborhood_sort_cluster = toronto_grouped.drop(["Neighborhood"], axis=1)
km = KMeans(init="k-means++",n_clusters=5, random_state=0).fit(neighborhood_sort_cluster)
cluster = pd.DataFrame({"Cluster":list(km.labels_)})
neighbor_list = []
for x in neighborhood_sort["Neighborhood"]:
    neighbor_list.append(x)
df_toronto2 = df_toronto.set_index("Neighborhood").loc[neighbor_list,["Latitude", "Longitude"]].reset_index().drop(["Neighborhood"], axis=1)
toronto_merged = pd.concat([neighborhood_sort,cluster,df_toronto2], axis=1)

In [518]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters